In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #to ignore pandas warning
import numpy as np
import pandas as pd
from NeuralNetwork import *
from utils import *


## Data Processing

### Applying One-Hot Encoding and Shuffling

In [2]:
col_names = ["Class", "a1", "a2", "a3", "a4", "a5", "a6", "ID"]
tr_data = pd.read_csv("../monk+s+problems/monks-1.train", sep =" ",  names = col_names)
test_data = pd.read_csv("../monk+s+problems/monks-1.test", sep =" ",  names = col_names)

tr_data = process_monk_data(tr_data)
test_data = process_monk_data(test_data)

print(tr_data)

     Class        ID  a1_1  a1_2  a1_3  a2_1  a2_2  a2_3  a3_1  a3_2  a4_1  \
0        1   data_42     1     0     0     1     0     0     0     1     0   
1        0  data_157     0     1     0     1     0     0     1     0     0   
2        0  data_143     1     0     0     0     0     1     0     1     0   
3        0   data_93     1     0     0     0     1     0     0     1     0   
4        1  data_197     0     1     0     0     1     0     1     0     1   
..     ...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
119      1  data_212     0     1     0     0     1     0     1     0     0   
120      1  data_206     0     1     0     0     1     0     1     0     0   
121      1  data_414     0     0     1     0     0     1     0     1     1   
122      1  data_153     0     1     0     1     0     0     1     0     0   
123      0  data_144     1     0     0     0     0     1     0     1     0   

     a4_2  a4_3  a5_1  a5_2  a5_3  a5_4  a6_1  a6_2  
0       0

## Creating the Neural Network

In [3]:

'''net = NeuralNetwork()

n_inputs = tr_data.shape[1] - 2

net.add_input_layer(n_inputs)

net.add_hidden_layer(n_inputs, 4)

net.add_output_layer(4, 1)
'''

'net = NeuralNetwork()\n\nn_inputs = tr_data.shape[1] - 2\n\nnet.add_input_layer(n_inputs)\n\nnet.add_hidden_layer(n_inputs, 4)\n\nnet.add_output_layer(4, 1)\n'

## Training

### Training with Mean Square Error

In [4]:
hyperparams = {"epochs": 500,
               "eta": 0.5,
               "clip_value": 1000,
               "hid_act_fun": "tanh",
               "out_act_fun": "sigmoid",
               "cost_fun": "mse",
               "mb" : 16,
               "momentum" : 0.01}


#losses = net.train(tr_data, hyperparams)  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

#plot_loss(losses, cost_fun = hyperparams["cost_fun"])

### Training with Binary Cross Entropy

In [5]:
'''
#RESET THE NETWORK!!!

losses = net.train(tr_data_X, tr_data_Y, epochs=1000,
                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE
# print(net)
# print(net.input_layer.weights)

plot_loss(losses, cost_fun="b_ce")
'''

'\n#RESET THE NETWORK!!!\n\nlosses = net.train(tr_data_X, tr_data_Y, epochs=1000,\n                   eta=0.5, clip_value=None, hid_act_fun="tanh", out_act_fun="sigmoid", cost_fun="b_ce")  # TANH IS FASTER?? SCRITTO SULLE SLIDE\n# print(net)\n# print(net.input_layer.weights)\n\nplot_loss(losses, cost_fun="b_ce")\n'

# Test Monk 1

In [6]:
#best_eta = net.k_fold(3, tr_data, hyperparams, "eta", [0.01, 0.05, 0.1,0.5, 0.8, 1])
#print(best_eta)

In [7]:
grid = {
    "eta" : [0.01, 0.1, 0.2, 0.4, 0.5, 0.7, 0.8 ,0.9, 1],
    "mb" : [4,8,16,32, tr_data.shape[0]],
    "momentum" : [1, 0.99, 0.95,  0.9],
    "n_layers" : [1],
    "n_neurons" : [3,4,5],
    "epochs" : [400],
    "clip_value" : [None],
    "hid_act_fun" : ["tanh"],
    "out_act_fun" : ["sigmoid"],
    "cost_fun" : ["b_ce"],
    "ridge_lambda": [None],
    "lasso_lambda": [None],
    "linear_decay": [None],
    "eta0": [None],
    "max_steps": [None],
    "epochs_update": [None]}
    

search_space = get_search_space(grid)

print(len(search_space))
print(search_space)
grid_results = parallel_grid_search(5, tr_data, search_space, n_inputs = tr_data.shape[1] - 2, n_outputs=1)

print(grid_results)
'''
'''


540
[{'eta': 0.01, 'mb': 4, 'momentum': 1, 'n_layers': 1, 'n_neurons': 3, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, {'eta': 0.01, 'mb': 4, 'momentum': 1, 'n_layers': 1, 'n_neurons': 4, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, {'eta': 0.01, 'mb': 4, 'momentum': 1, 'n_layers': 1, 'n_neurons': 5, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}, {'eta': 0.01, 'mb': 4, 'momentum': 0.99, 'n_layers': 1, 'n_neurons': 3, 'epochs': 400, 'clip_value': None, 

end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
end Training
Valid error:[0.08769678] with par {'eta': 0.9, 'mb': 124, 'momentum': 0.95, 'n_layers': 1, 'n_neurons': 3, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lambda': None, 'lasso_lambda': None, 'linear_decay': None, 'eta0': None, 'max_steps': None, 'epochs_update': None}
end Training
end Training
Valid error:[nan] with par {'eta': 0.8, 'mb': 124, 'momentum': 1, 'n_layers': 1, 'n_neurons': 3, 'epochs': 400, 'clip_value': None, 'hid_act_fun': 'tanh', 'out_act_fun': 'sigmoid', 'cost_fun': 'b_ce', 'ridge_lamb

'\n'

In [3]:
params = {
    "eta" : 0.2,
    "mb" : 1,
    "momentum" : 0.6,
    "n_layers" : 1,
    "n_neurons" : 4,
    "epochs" : 400,
    "clip_value" : None,
    "hid_act_fun" : "tanh",
    "out_act_fun" : "sigmoid",
    "cost_fun" : "mse",
    "ridge_lambda": None,
    "lasso_lambda": None,
    "linear_decay": None,
    "eta0": None,
    "max_steps": None,
    "epochs_update": None}

best_net, var, bias = compare_models(2, tr_data, params, n_inputs = tr_data.shape[1] - 2, n_outputs=1)


HIDDEN LAYER: NODE 0 WEIGHTS = 0.20072454723850017, 0.6455742154436642, 0.14836141155555155, -0.03535008105049931, 0.1111274662180163, -0.38268040064369946, -0.23522575553583452, -0.5867479674988183, 0.177247427977101, 0.5086936903155551, 0.0750848874061312, 0.5436555946116127, 0.14858358985381812, -0.25681506069019583, 0.5595742304011939, 0.20826723919011647, 0.2853059686846552,  BIAS = 0.0
NODE 1 WEIGHTS = 0.012872912000377752, 0.2168188189274789, -0.23270561017050623, -0.5408403615616199, 0.25136637086033997, -0.12200992189387842, 0.0970165494835567, -0.3462035748162753, 0.39028561307279697, -0.7203953928106612, -0.06487325198911238, -0.153844978715895, -0.11855509766140948, -0.1293872583179218, 0.15819765502384553, -0.26404299054753705, 0.01818548188833722,  BIAS = 0.0
NODE 2 WEIGHTS = 0.13874682983869682, -0.2507724096236018, 0.016603776995886776, 0.09152175360012045, -0.38234452395188495, 0.16789695342853136, 0.49690756924404306, 0.11046811625536894, 0.3847136698566844, 0.3455678

In [4]:
print(best_net)
print(var)
print(bias)

INPUT LAYER: 
NODE 0 OUTPUT = 0
NODE 1 OUTPUT = 0
NODE 2 OUTPUT = 1
NODE 3 OUTPUT = 0
NODE 4 OUTPUT = 0
NODE 5 OUTPUT = 1
NODE 6 OUTPUT = 1
NODE 7 OUTPUT = 0
NODE 8 OUTPUT = 0
NODE 9 OUTPUT = 0
NODE 10 OUTPUT = 1
NODE 11 OUTPUT = 0
NODE 12 OUTPUT = 0
NODE 13 OUTPUT = 0
NODE 14 OUTPUT = 1
NODE 15 OUTPUT = 0
NODE 16 OUTPUT = 1

LAYER 0 
NODE 0 WEIGHTS = 0.27911451223453027, -0.2654075138884794, -0.0683040149144846, 0.672240020850368, -0.4321965209430798, 0.5070139819123344, 0.22578330297916963, -0.16683183275108188, -0.12001588906500137, -0.19101425155878565, 0.6086341104932276, 0.5453522315278774, 0.6847667452002484, -0.37710323176260757, -0.6637161737560301, -0.16509162655711013, -0.0218775635764175,  BIAS = 0.0
NODE 1 WEIGHTS = -0.27832848135747856, 0.27238292262594915, 0.05265195490486636, -0.03133687756668484, -0.34117115337247034, 0.46601121819241514, 0.12211776807069598, -0.19440239153338493, 0.3015987630829442, -0.22705197173514183, -0.21090485923566918, -0.3670997721953509, -0.1